## Overview

The Gemini File API provides a simple way for developers to upload files and use them with the Gemini API in multimodal scenarios. This notebook shows how to use the File API to upload an image and include it in a `GenerateContent` call to the Gemini API.

Note: This API is currently in [private preview](https://github.com/google-gemini/gemini-api-cookbook/blob/main/preview/README.md) and is [only available in certain regions](https://ai.google.dev/available_regions).


## Setup


### Install the Gemini API Python SDK

In [ ]:
# TODO (tymichaelchen@): Use PyPI package once version is released.
!pip install -U git+https://github.com/google/generative-ai-python # Install the Python SDK

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Upload a file to the File API

The File API lets you upload a variety of multi-modal MIME types including images. The File API is only intended as input to generate content and has the following attributes:

* Can only be used with [`model.generateContent`](https://ai.google.dev/api/rest/v1beta/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1beta/models/streamGenerateContent)
* Automatic file deletion after 2 days
* Maximum 2GB per file, 20GB limit per project
* No downloads allowed

First, click **Files** on the left sidebar, then click **Upload** and upload an image file to Colab.

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

Next, we'll upload that file to the File API. In the form for the code cell below, enter the filename for the file you uploaded and provide an appropriate display name for the file, then run the cell.

In [ ]:
# Implement the File class expected by the File API
class File:
  def __init__(self, file_path: str, display_name: str = None,
               response = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name

  def set_response(self, response):
    self.response = response

# Specify the file to upload
my_filename = "gemini_logo.png" # @param {type:"string"}
my_file_display_name = "Gemini Logo" # @param {type:"string"}

# Create a File
uploaded_file = File(
    file_path=my_filename,
    display_name=my_file_display_name
)

# Upload the file to the File API
response = genai.upload_file(path=uploaded_file.file_path,
                             display_name=uploaded_file.display_name)
uploaded_file.set_response(response)
print(f"Uploaded file {response.display_name} as: {response.uri}")

## Get File

`files.get` lets you get the file metadata that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique. Only use the `displayName` to identify files if you manage uniqueness yourself.

In [ ]:
# Get files uploaded in the API
response = genai.get_file(name=response.name)
print(f"Retrieved file {response.display_name} as: {response.uri}")

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI. In this example, `makeGenerateContentRequest()` creates a request that starts with a text `Part` and then has a series of image file `Part`s, but valid `Part`s can come in any order.

In [ ]:
# Make GenerateContent LLM call
prompt = "Describe the image with a creative description" # @param {type:"string"}
model_name = "models/gemini-1.5-pro-latest" # @param ["models/gemini-1.5-pro-latest", "models/gemini-1.0-pro-vision-latest"]

model = genai.GenerativeModel(model_name=model_name)
response = model.generate_content([prompt, uploaded_file.response])
print(response)

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
# Delete the files with its resource name
# TODO(tymichaelchen@): Confirm this works after the colab fix.
genai.delete_file(uploaded_file.response.name)
print(f'Deleted {uploaded_file.display_name}')